In [1]:
import langchain 
print(langchain.__version__)

0.0.248


In [2]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://mazcaeanclabopenai01.openai.azure.com"


In [3]:
# # Set this to `azure`
# export OPENAI_API_TYPE=azure
# # The API version you want to use: set this to `2023-05-15` for the released version.
# export OPENAI_API_VERSION=2023-05-15
# # The base URL for your Azure OpenAI resource. You can find this in the Azure portal under your Azure OpenAI resource.
# export OPENAI_API_BASE=https://your-resource-name.openai.azure.com  
# # The API key for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
# export OPENAI_API_KEY=<your Azure OpenAI API key>

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter,TokenTextSplitter
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
import magic
import os
import nltk

In [5]:
from langchain.document_loaders import PyPDFLoader

loader =  PyPDFLoader("https://www.rogers.com/cms/pdf/en/Consumer_SUG_V20.pdf")
# Load up your text into documents
data = loader.load()

In [6]:
from transformers import AutoTokenizer, AutoModel

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [7]:
#customize splitter which count token length instead of length, control the tokens to be less than 230
#(256 as the upper limit of the embedding model)
def huggingface_tokenizer_length(text: str) -> int:
    return len(tokenizer.encode(text))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=230, chunk_overlap=20,length_function=huggingface_tokenizer_length)
# Split your documents into texts
texts = text_splitter.split_documents(data)

In [8]:
for i in texts[:10]:
    print(len(i.page_content))
print("_____")
for i in texts[:10]:
    print(huggingface_tokenizer_length(i.page_content))

445
595
666
774
586
666
815
731
209
836
_____
116
138
155
166
127
152
164
156
47
190


In [11]:
from langchain.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(deployment='text-embedding-ada-002',
                                model='text-embedding-ada-002',
                                chunk_size=8)

In [12]:
# # Turn your texts into embeddings
# #By default, input text longer than 384 word pieces is truncated.
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': False}
# embeddings = HuggingFaceEmbeddings(
#                 model_name=model_name,
#                 model_kwargs=model_kwargs,
#                 encode_kwargs=encode_kwargs
#             )

In [13]:
# Get your docsearch ready
from langchain.vectorstores import Chroma
docsearch = Chroma.from_documents(texts, embeddings)

In [11]:
docsearch

In [14]:
from langchain.prompts import PromptTemplate
prompt_template = """
        You are a customer service agent of Rogers Communications which is a telecommunication company in Canada.
        You are helpful and honest on answering customers' questions.  

        Use the following information about Rogers Communications to respond to 
        the customer's query acting as a customer service agent.
        Context: {context}                                

        Keep the response brief and only use the information given by the context. If you don't know the answer, just say that you don't know, don't try to make up an answer.

        Customer: {question}
        Customer service agent:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [15]:
from langchain.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4-32k",
    model_name="gpt-4-32k"
)

In [16]:
print(llm)

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-4-32k' temperature=0.7 model_kwargs={} openai_api_key='27c9d423022649f1976f8a36029deea4' openai_api_base='https://mazcaeanclabopenai01.openai.azure.com' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=None deployment_name='gpt-4-32k' openai_api_type='azure' openai_api_version='2023-05-15'


In [17]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                 retriever=docsearch.as_retriever(search_kwargs={"k": 2}), 
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=True)

In [18]:
# Run a query
queries = [
        "how to download ringtones?",
        "How are my services invoiced for the first bill?"
]

query_1 = queries[0]
result_1 = qa({"query": query_1})

In [19]:
result_1['result']

'To download ringtones, please follow these steps:\n\n1. Select the Internet icon from your main menu and then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.\n2. Check out categories such as Top Sellers and Spotlights by selecting Ringtones.\n3. Pick a ringtone of your choice, then click on the Download link. Once downloaded, an option will appear to set the new download as your ringtone.'

In [20]:
result_1['source_documents']

[Document(page_content='to the original point of purchase.Mobile Backup\nHARdw ARe RetuRN PoliCy\nGet the most out of each battery charge \nso your phone is ready whenever you are.Tip BAtteRy PeRfoRMANCe  VAlue \nPACksincluded inVAlue \nPACksincluded in\n1  Select the Internet icon \nfrom your main menu. \nThen select the Shop link from your mobile internet home page or text MUSIC  \nto 555 for a link to the \nMusic and Tones page.\n2  Select Ringtones to check \nout categories such as Top Sellers and Spotlights.\n3   Select a ringtone, then click on the Download  \nlink. Once downloaded, an option will appear to set the new download as your ringtone.how\nto\n04/05For details, applicable conditions and a list of supported \ndevices, visit rogers.com/mobilebackup', metadata={'page': 2, 'source': 'C:\\Users\\YUNCHE~1.LIA\\AppData\\Local\\Temp\\tmpiuu159fn\\tmp.pdf'}),
 Document(page_content='the Ringbacks mobile store.\nClick the link.\n3   Now you can browse to download your Ringback.\n

In [21]:
query_2 = queries[1]
result_2 = qa({"query": query_2})
result_2['result']

'Your first bill includes both partial and regular charges. The regular charges include your monthly plan and any additional fees that are billed in advance for the upcoming month. Partial charges are calculated based on the length of time between your activation date and your invoice date. This covers your usage for that period. For example, if you activated a $30 plan with 150 minutes on May 5, you’d have 40 minutes of airtime to use until May 12 and would see a partial charge of about $8 for those 8 days of service.'

In [22]:
result_2['source_documents']

[Document(page_content='08Your first bill\nLearn how billing works, in plain language\nFor example, if you activated a $30 plan with 150 minutes on May 5, you’d have 40 minutes of airtime to use until May 12 and would see a partial charge of approximately $8 for your rate plan for the 8 days of service.First bill explanation\nHow AR e My seRViCes iNVoiCed?\nYour first bill includes partial charges and regular charges. Regular charges include your monthly plan and any add-on fees billed in advance for the next month. If you exceed monthly usage limits, extra charges will be listed on your next invoice in the “Regular Charges” section.\nwHy AR e tHeRe PAR tiAl CHARG es oN My \nBill, AN d How AR e tHey CA lCulAted?', metadata={'page': 4, 'source': 'C:\\Users\\YUNCHE~1.LIA\\AppData\\Local\\Temp\\tmpiuu159fn\\tmp.pdf'}),
 Document(page_content='Bill, AN d How AR e tHey CA lCulAted?\nYour activation date and billing period don’t always start on the same date, so your allotted usage is pro-ra

### Sources

In [18]:
import os
#OPENAI_API_KEY = 'sk-rTu5MoMgXzRJXo7gMfL##########cT3BlbkFJL6Nd4rdFAEraQStnh9G7'
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
openai_llm=OpenAI(temperature=0)

In [23]:
openai_qa = RetrievalQA.from_chain_type(llm=openai_llm, chain_type="stuff", 
                                 retriever=docsearch.as_retriever(search_kwargs={"k": 2}), 
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=False)

In [24]:
openai_result_1 = openai_qa({"query": query_1})
openai_result_1['result']

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 10.0 seconds as it raised RateLimitError: You exceeded your current quota, please ch

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [ ]:
openai_result_1['source_documents'] 

' McCarthy discovered a way to build a whole programming language using a handful of simple operators and a notation for functions. He called this language Lisp, for "List Processing," because one of his key ideas was to use a simple data structure called a list for both code and data.'

[Document(page_content='May 2001\n\n(I wrote this article to help myself understand exactly\n\nwhat McCarthy discovered.  You don\'t need to know this stuff\n\nto program in Lisp, but it should be helpful to\n\nanyone who wants to\n\nunderstand the essence of Lisp \x97 both in the sense of its\n\norigins and its semantic core.  The fact that it has such a core\n\nis one of Lisp\'s distinguishing features, and the reason why,\n\nunlike other languages, Lisp has dialects.)In 1960, John\n\nMcCarthy published a remarkable paper in\n\nwhich he did for programming something like what Euclid did for\n\ngeometry. He showed how, given a handful of simple\n\noperators and a notation for functions, you can\n\nbuild a whole programming language.\n\nHe called this language Lisp, for "List Processing,"\n\nbecause one of his key ideas was to use a simple\n\ndata structure called a list for both\n\ncode and data.It\'s worth understanding what McCarthy discovered, not\n\njust as a landmark in the histo